# 1. Deep Supervised Hashing
While brushing up on algorithms I started to wonder if there was any research into estimating hash functions with Deep Learning. I discovered there has been a lot of interesting research, with much of it covered in [A Survey of Deep Hashing Methods (2022)](https://arxiv.org/pdf/2003.03369.pdf). In this notebook I implemented one supervised pairwise deep hashing methodology described in [Deep Supervised Hashing for Fast Image Retrieval (2016)](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Liu_Deep_Supervised_Hashing_CVPR_2016_paper.pdf). The original model was developed using Caffe - how times have changed! In this notebook I use Tensorflow and make some minor changes to the original methodology, described below. Also I analyze the MNIST dataset to keep the analysis quick, since the primary purpose is to improve my understanding of image hashing and not push the limits of my GPU.

In [310]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from scipy.spatial.distance import hamming

mnist = datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train/255, X_test/255
X_train = tf.expand_dims(X_train, axis=-1)
X_test = tf.expand_dims(X_test, axis=-1)

# 2. Model structure
I use a model structure similar to the one described in the paper. Since the MNIST I'm analyzing is simpler than CIFAR-10 data used in the paper my model contains one fewer convolutional layer and fewer filters per layer. Additionally my first fully connected layer has only 128 nodes and I generate 17 outputs - enough to encode roughly twice the number of images in my sample. Finally I use tanh activation for the final layer to approximate the quantization step. In total this model contains 9,137 parameters.

In [317]:
model = models.Sequential()
model.add(layers.Conv2D(12, (5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(8, (5, 5), activation='relu'))
model.add(layers.AveragePooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(17, activation='tanh'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 24, 24, 12)        312       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 8, 8, 12)          0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 4, 4, 8)           2408      
_________________________________________________________________
average_pooling2d_12 (Averag (None, 2, 2, 8)           0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_25 (Dense)             (None, 17)              

# 3. Estimation methodology
I use the loss function described in the paper, except without regularization. For each batch I calculate the pairwise distance between each pair of hashes. If a pair of hashes shares a label, the distance should be low. If a pair of hashes have different labels, the distance should be high. Also the magnitude of the similarity between labels should be reflected in their respective hashes.

I run 10 training epochs using the adam optimizer and a batch size of 128.

In [318]:
def deephashloss(y_true, y_pred):
    m = 34
    y_true = tf.cast(tf.equal(y_true, tf.transpose(y_true)), y_pred.dtype)
    D = tf.reduce_sum((tf.expand_dims(y_pred, 1)-tf.expand_dims(y_pred, 0))**2,2) # pairwise distances
    hashloss = tf.reduce_sum(tf.math.multiply(y_true, D) + tf.math.multiply(1-y_true, tf.maximum(m-D,0)), axis=1)
    return hashloss*0.5

model.compile(optimizer='adam', loss=deephashloss)
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
469/469 [==============================] - 3s 5ms/step - loss: 576.5067 - val_loss: 383.5576
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 361.9986 - val_loss: 331.0095
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 316.3627 - val_loss: 280.7243
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 266.4889 - val_loss: 232.8251
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 219.8488 - val_loss: 193.5880
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 176.7694 - val_loss: 152.6449
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 150.4059 - val_loss: 135.7185
Epoch 8/10
469/469 [==============================] - 2s 4ms/step - loss: 133.3176 - val_loss: 119.4876
Epoch 9/10
469/469 [==============================] - 2s 4ms/step - loss: 119.4915 - val_loss: 114.5700
Epoch 10/10
469/469 [==============================] - 2s 4ms/st

# 4. Model performance
To evaluate model performance I evaluate the hashes among labels in the test sample. First I calculate the median hash within each label. Then I calculate the hamming distance between each pair of labels. I find that the hashes for images containing the number "4" and those containing the number "9" have the smallest hamming distance of 2. While those containing the number "0" and the number "1" have the highest hamming distance of 15. These results are consistent with my expectations about the relative similarity between the images of these numbers.

In [321]:
y_test_fit = model.predict(X_test)
y_test_fit = np.where(y_test_fit>0,1,0)

# compute median hash code for each label
median_hash = [np.median(y_test_fit[y_test==n],axis=0) for n in range(10)]

hash_distance = {}
for i in range(10):
    for j in range(i+1,10):
        hash_distance[(i,j)] = hamming(median_hash[i],median_hash[j])*len(median_hash[i])

print("Hamming distance", max(hash_distance,key=hash_distance.get), "=", max(hash_distance.values()))
print("Hamming distance", min(hash_distance,key=hash_distance.get), "=", min(hash_distance.values()))

Hamming distance (0, 1) = 15.0
Hamming distance (4, 9) = 2.0
